<a target="_blank" href="https://colab.research.google.com/github/PassengerSim/algorithms/blob/main/forecasting/conditional-forecasting.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>  

This notebook defines algorithms for Q forecasting conditional on availability,
also now known as "conditional forecasting".

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Exogenous Inputs to Conditional Forecast

## Timeframes

The booking horizon is divided into timeframes of arbitrary length, not necesssarily 
homogeneous. Without loss of generality, we assume each timeframe is a whole number of 
days. Each timeframe can be identified by a `DCP_INDEX`, which starts at 0 and increases
by one sequentially through the timeframes, or by a `TF` identity, which gives the
number of days from departure at the beginning of the timeframe.


In [2]:
timeframes = pd.Series([21, 14, 7], name="TF").rename_axis(index="DCP_INDEX")
timeframes.to_frame()

,TF
DCP_INDEX,
0,21
1,14
2,7


## Frat5 Curve

This curve defines, for each timeframe, the fare ratio at which half of
customers would be willing to buy up to a higher fare class.


In [3]:
frat_5_curve = pd.Series([1.20, 1.63, 2.83], index=timeframes, name="FRAT5")
frat_5_curve.to_frame()

,FRAT5
TF,
21,1.20
14,1.63
7,2.83


## Fare Classes and Prices

In [4]:
fares = pd.Series(
    {
        "Y0": 500,
        "Y1": 400,
        "Y2": 300,
        "Y3": 225,
        "Y4": 175,
        "Y5": 150,
    },
    name="PRICE",
).rename_axis(index="FARECLASS")

assert fares.is_monotonic_decreasing

fares.to_frame()

,PRICE
FARECLASS,
Y0,500
Y1,400
Y2,300
Y3,225
Y4,175
Y5,150


## Advance Purchace Restrictions

These restriction close fare classes by a rule once the days prior to departure drops
below a threshold limit.

In [5]:
fare_ap = {"Y5": 14, "Y4": 7}

## Sales and Closure History

We presume we have recorded historical sales by fare class for 26 prior 
sampled days. We also have recorded whether each fare was available
for sale or not, so we can differentiate two differnt zero sales states:
because the fare class was not available, or because it was available
but we simply failed to sell it.

Conditional forecasting assumes a fully fenceless/unrestricted market, 
with the (optional) exception of advance purchase restrictions, which
apply to the time of purchase but do not otherwise differentiate fare
classes.  In this environment, there will theoretically be no sales 
above the lowest available fare class at any moment, as there is no 
reason for any customer to purchase a fare class higher than the 
lowest (least expensive) available.  In practice, sometimes "stuff happens"
and other fare classes somehow end up getting sold, but we ignore this 
for simulation. 

In this example, the Y5 class has an AP restriction at 14 days (i.e. after
the first timeframe) and the Y4 class has an AP restriction at 7 days 
(i.e. after the second timeframe).

For ease of exposition in this notebook, we comingle the sales and closure data 
here into one data structure, indicating a closure with X and a number of sales
with an integer, so that "0" means we could have sold a fare being offered but
did not, and "X" means the fare was not offered (and thus never sold).


In [6]:
# some tools for working with the data

X = np.nan


def format_sales_and_closure(**raw_sales):
    """Convert raw sales data to a sales and closure dataframes."""
    sales = pd.concat(
        {
            k: pd.DataFrame(v)
            .rename_axis(index="SAMPLE", columns="TF")
            .rename(columns=timeframes)
            for k, v in raw_sales.items()
        },
        names=["FARECLASS"],
    )
    closures = sales.isna()
    sales = sales.fillna(0).astype(int)

    # check closures are ordered consistent with monotonicity rules,
    # so that the closure of any fare class in any timeframe requires
    # the closure of all less expensive fare classes in the same timeframe
    assert (closures.loc["Y5"] >= closures.loc["Y4"]).all(axis=None)
    assert (closures.loc["Y4"] >= closures.loc["Y3"]).all(axis=None)
    assert (closures.loc["Y3"] >= closures.loc["Y2"]).all(axis=None)
    assert (closures.loc["Y2"] >= closures.loc["Y1"]).all(axis=None)
    assert (closures.loc["Y1"] >= closures.loc["Y0"]).all(axis=None)

    return sales, closures

In [7]:
# Clean Sales Data
#
# This sales history represents a scenario where all sales are made at the
# lowest available fare class, and the identified of the lowest fare class
# changes only at DCPs and not in the middle of a timeframe.  This is an
# artificial construct applied here to simplify the exposition for a base
# algorithm.

# fmt: off

sales_clean, closures_clean = format_sales_and_closure(
    Y5=[
        [ 7, X, X],  # 00
        [ 7, X, X],  # 01
        [ 6, X, X],  # 02
        [ X, X, X],  # 03
        [19, X, X],  # 04
        [ X, X, X],  # 05
        [11, X, X],  # 06
        [ X, X, X],  # 07
        [17, X, X],  # 08
        [ 8, X, X],  # 09
        [ X, X, X],  # 10
        [23, X, X],  # 11
        [28, X, X],  # 12
        [17, X, X],  # 13
        [13, X, X],  # 14
        [ X, X, X],  # 15
        [11, X, X],  # 16
        [18, X, X],  # 17
        [ X, X, X],  # 18
        [ X, X, X],  # 19
        [ X, X, X],  # 20
        [ X, X, X],  # 21
        [ X, X, X],  # 22
        [ X, X, X],  # 23
        [ X, X, X],  # 24
        [ X, X, X],  # 25
    ],
    Y4=[
        [0, 1, X],  # 00
        [0, 1, X],  # 01
        [0, 1, X],  # 02
        [3, 1, X],  # 03
        [0, 2, X],  # 04
        [5, 1, X],  # 05
        [0, 1, X],  # 06
        [6, 2, X],  # 07
        [0, 1, X],  # 08
        [0, 1, X],  # 09
        [9, X, X],  # 10
        [0, 1, X],  # 11
        [0, X, X],  # 12
        [0, 1, X],  # 13
        [0, 2, X],  # 14
        [4, X, X],  # 15
        [0, 2, X],  # 16
        [0, 1, X],  # 17
        [0, 2, X],  # 18
        [0, 1, X],  # 19
        [0, 1, X],  # 20
        [9, 2, X],  # 21
        [0, 1, X],  # 22
        [X, X, X],  # 23
        [0, 1, X],  # 24
        [X, X, X],  # 25
    ],
    Y3=[
        [0, 0, 1],  # 00
        [0, 0, 2],  # 01
        [0, 0, 0],  # 02
        [0, 0, 1],  # 03
        [0, 0, X],  # 04
        [0, 0, 1],  # 05
        [0, 0, 3],  # 06
        [0, 0, 2],  # 07
        [0, 0, 1],  # 08
        [0, 0, 2],  # 09
        [0, 1, 3],  # 10
        [0, 0, 1],  # 11
        [0, 5, X],  # 12
        [0, 0, 1],  # 13
        [0, 0, 2],  # 14
        [0, 2, 1],  # 15
        [0, 0, 1],  # 16
        [0, 0, X],  # 17
        [0, 0, X],  # 18
        [0, 0, 1],  # 19
        [0, 0, X],  # 20
        [0, 0, X],  # 21
        [0, 0, 0],  # 22
        [X, X, 1],  # 23
        [0, 0, X],  # 24
        [4, 4, 1],  # 25
    ],
    Y2=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, X],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, X],  # 17
        [0, 0, 1],  # 18
        [0, 0, 0],  # 19
        [0, 0, 2],  # 20
        [0, 0, 1],  # 21
        [0, 0, 0],  # 22
        [1, 0, 0],  # 23
        [0, 0, 1],  # 24
        [0, 0, 0],  # 25
    ],
    Y1=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 1],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 1],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
    ],
    Y0=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 1],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
    ],
)

# fmt: on

In [8]:
# Heavy Sales Data
#
# This sales history represents a scenario where all sales are made at the
# lowest available fare class, and the identified of the lowest fare class
# changes only at DCPs and not in the middle of a timeframe.  This is an
# artificial construct applied here to simplify the exposition for a base
# algorithm.  In addition, in every timeframe, there is at least one sale
# in the lowest available fare class (i.e. no zeros anywhere that there
# might not be a zero).  This scenario is used for some technical diagnostics
# and is not a realistic scenario.

# fmt: off

sales_heavy, closures_heavy = format_sales_and_closure(
    Y5=[
        [ 7, X, X],  # 00
        [ 7, X, X],  # 01
        [ 6, X, X],  # 02
        [ X, X, X],  # 03
        [19, X, X],  # 04
        [ X, X, X],  # 05
        [11, X, X],  # 06
        [ X, X, X],  # 07
        [17, X, X],  # 08
        [ 8, X, X],  # 09
        [ X, X, X],  # 10
        [23, X, X],  # 11
        [28, X, X],  # 12
        [17, X, X],  # 13
        [13, X, X],  # 14
        [ X, X, X],  # 15
        [11, X, X],  # 16
        [18, X, X],  # 17
        [ X, X, X],  # 18
        [ X, X, X],  # 19
        [ X, X, X],  # 20
        [ X, X, X],  # 21
        [ X, X, X],  # 22
        [ X, X, X],  # 23
        [ X, X, X],  # 24
        [ X, X, X],  # 25
    ],
    Y4=[
        [0, 1, X],  # 00
        [0, 1, X],  # 01
        [0, 1, X],  # 02
        [3, 1, X],  # 03
        [0, 2, X],  # 04
        [5, 1, X],  # 05
        [0, 1, X],  # 06
        [6, 2, X],  # 07
        [0, 1, X],  # 08
        [0, 1, X],  # 09
        [9, X, X],  # 10
        [0, 1, X],  # 11
        [0, X, X],  # 12
        [0, 1, X],  # 13
        [0, 2, X],  # 14
        [4, X, X],  # 15
        [0, 2, X],  # 16
        [0, 1, X],  # 17
        [3, 2, X],  # 18
        [2, 1, X],  # 19
        [4, 1, X],  # 20
        [9, 2, X],  # 21
        [2, 1, X],  # 22
        [X, X, X],  # 23
        [1, 1, X],  # 24
        [X, X, X],  # 25
    ],
    Y3=[
        [0, 0, 1],  # 00
        [0, 0, 2],  # 01
        [0, 0, 3],  # 02
        [0, 0, 1],  # 03
        [0, 0, X],  # 04
        [0, 0, 1],  # 05
        [0, 0, 3],  # 06
        [0, 0, 2],  # 07
        [0, 0, 1],  # 08
        [0, 0, 2],  # 09
        [0, 1, 3],  # 10
        [0, 0, 1],  # 11
        [0, 5, X],  # 12
        [0, 0, 1],  # 13
        [0, 0, 2],  # 14
        [0, 2, 1],  # 15
        [0, 0, 1],  # 16
        [0, 0, X],  # 17
        [0, 0, X],  # 18
        [0, 0, 1],  # 19
        [0, 0, X],  # 20
        [0, 0, X],  # 21
        [0, 0, 1],  # 22
        [X, X, 1],  # 23
        [0, 0, X],  # 24
        [4, 4, 1],  # 25
    ],
    Y2=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, X],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, X],  # 17
        [0, 0, 1],  # 18
        [0, 0, 0],  # 19
        [0, 0, 2],  # 20
        [0, 0, 1],  # 21
        [0, 0, 0],  # 22
        [1, 2, 0],  # 23
        [0, 0, 1],  # 24
        [0, 0, 0],  # 25
    ],
    Y1=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 1],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 1],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
    ],
    Y0=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 1],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
    ],
)

# fmt: on

assert sales_heavy.groupby("SAMPLE").sum().min(axis=None) > 0

In [9]:
# Select Scenario for Notebook
sales, closures = sales_clean, closures_clean

In [10]:
n_samples = len(sales.index.get_level_values("SAMPLE").unique())
n_samples

26

## Lowest Available Class

For each timeframe/sample, we identify the index of the lowest fare class available.

In [11]:
top_class = fares.index[0]

lowest_available_class = pd.DataFrame(
    0, columns=closures.loc[top_class].columns, index=closures.loc[top_class].index
)
for class_number, fare_class in enumerate(fares.index[1:], start=1):
    lowest_available_class = lowest_available_class.where(
        closures.loc[fare_class], class_number
    )

lowest_available_class

TF,21,14,7
SAMPLE,,,
0,5,4,3
1,5,4,3
2,5,4,3
3,4,4,3
4,5,4,1
5,4,4,3
6,5,4,3
7,4,4,3
8,5,4,3


In [12]:
# check data is clean (no sales above lowest available class)
any_sales = sales.groupby("SAMPLE").max() > 0
highest_sold_fareclass = sales.groupby("SAMPLE").idxmax().map(lambda x: int(x[0][1:]))
assert not (highest_sold_fareclass[any_sales] - lowest_available_class).any(axis=None)

# Calculated Values

In [13]:
def supc(t):
    return -np.log(0.5) / (t - 1)


supc(frat_5_curve)

TF
21    3.465736
14    1.100234
7     0.378769
Name: FRAT5, dtype: float64

In [14]:
def outer_product(s1: pd.Series, s2: pd.Series):
    """Compute the outer product of two Series as a DataFrame."""
    return pd.DataFrame(np.outer(s1, s2), index=s1.index, columns=s2.index).rename_axis(
        index=s1.index.name, columns=s2.index.name
    )

### Sellup Probability

The sellup probability starts with a customer who would definitely purchase the bottom 
fare class if it is available.  It then computes the probability that, for any fare 
class that might be the lowest currently available, what is the probability that the
customer would purchase that fare class.  So, for the bottom class, the value is 1.0,
while for progressively higher priced fares the value drops.  This sellup probability
can be computed from the Frat5 value (or other sellup computation) and the list of
all possible fare prices.

In [15]:
def sellup_prob_func(fares, frat_5_curve):
    minimum_fare = fares.min()
    df = outer_product(
        -((fares / minimum_fare) - 1),
        supc(frat_5_curve),
    )
    return np.exp(df)


sellup_prob = sellup_prob_func(fares, frat_5_curve)
sellup_prob

TF,21,14,7
FARECLASS,,,
Y0,0.000308,0.076749,0.413212
Y1,0.003100,0.159818,0.531910
Y2,0.031250,0.332793,0.684704
Y3,0.176777,0.576882,0.827468
Y4,0.561231,0.832458,0.938823
Y5,1.000000,1.000000,1.000000


### Net Sellup Probability

The net sellup probability transforms the sellup probability, expressing the
fraction of customers in any timeframe who would be expected to purchase a 
given fare class but *not* anything more expensive.  Put another way, it is
the probability of losing the sale if you close a particular fare class. 

In [16]:
def net_sellup_prob_func(fares, frat_5_curve):
    """Differences in sellup rates."""
    differences = sellup_prob.diff(axis=0)
    # use fillna, to set the values of the top fare class
    # equal to their gross values (i.e. diff from zero)
    return differences.fillna(sellup_prob)


net_sellup_prob = net_sellup_prob_func(fares, frat_5_curve)
net_sellup_prob

TF,21,14,7
FARECLASS,,,
Y0,0.000308,0.076749,0.413212
Y1,0.002793,0.083068,0.118698
Y2,0.028150,0.172976,0.152794
Y3,0.145527,0.244089,0.142765
Y4,0.384454,0.255576,0.111355
Y5,0.438769,0.167542,0.061177


### KI fare adjustment


In [17]:
def fare_adj_ki_func(sellup_prob, fares, scale_factor=1):
    """Fare adjustment using KI algorithm."""
    df = sellup_prob.mul(fares, axis=0)
    df = df.diff().div(sellup_prob.diff())
    df = df.T.fillna(fares).T
    if scale_factor != 1:
        df = df.mul(scale_factor).add(fares * (1 - scale_factor), axis=0)
    return df


fare_adj_ki = fare_adj_ki_func(sellup_prob, fares)
fare_adj_ki

TF,21,14,7
FARECLASS,,,
Y0,500.000000,500.000000,500.000000
Y1,388.986018,307.607025,51.878172
Y2,288.986018,207.607025,-48.121828
Y3,208.894707,122.744306,-134.702735
Y4,152.009402,62.140631,-196.545717
Y5,118.022407,25.783507,-233.651297


# Conditional Forecast (Clean)

The "conditional" forecast is just that: a forecast that is conditioned on the lowest 
available fare class.

### Expected Sales

The first step in the conditional forecast is to use the sellup probabilities to find the 
expected (fractional) number of sales per unit demand that would have occurred in each
historical timeframe, based on the recorded lowest available fare class from each
historical timeframe.

In [18]:
def expected_sales_func(lowest_available_class):
    """Expected bookings per unit of Q demand."""

    # initialize an expected bookings DataFrame with zeros
    expected_bookings = pd.DataFrame(0.0, columns=sales.columns, index=sales.index)

    # compute expected bookings from lowest available class and sellup probabilities
    for samp_num in range(n_samples):
        for tf in frat_5_curve.index:
            class_num = lowest_available_class.loc[samp_num, tf]
            sup = sellup_prob.loc[f"Y{class_num}", tf]
            expected_bookings.loc[(f"Y{class_num}", samp_num), tf] = sup

    return expected_bookings


expected_sales_per_unit_demand = expected_sales_func(lowest_available_class)

Then, we can solve a linear regression for each time frame, to compute the 
mean level of "Q" demand in each timeframe.  Below are several options on
how to construct this linear regression model. To consider and contrast these 
options, first we will initialize dictionaries to hold various results.

In [19]:
q_means = {}
q_variances = {}

### Option 1: Unweighted Linear Regression

This regression gives equal weight to each unit of demand, 
no matter in which fare class it is expected to show up.
This implicitly gives more weight to fitting demand at the 
lower fare classes, which have more activity when they are
open.

In [20]:
q_means[1] = np.zeros(len(timeframes))
q_variances[1] = np.zeros(len(timeframes))

for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales.iloc[:, i].to_frame(),
    )
    q_means[1][i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_variances[1][i] = mean_squared_error(sales.iloc[:, i], y) * len(fares)

print("Forecast Mean:", q_means[1])
print("Fcst Variance:", q_variances[1])


Forecast Mean: [12.48363676  1.83805027  1.6379777 ]
Fcst Variance: [33.811018    1.19089358  0.5034093 ]


### Option 2: LR Weighted by Inverse Sellup Probability

This regression normalized weights based on the sellup probability, 
counting more heavily the demand that survives when forced
to buy at a higher price. This gives more weight to fitting 
demand at the higher fare classes, which have less activity 
but are more meaningful when they are active.

Without some kind of cap, this can potentially end up giving large 
weights to high value fare classes in early timeframes, but no cap
is envisioned in this example code.

In [21]:
weights = 1 / sellup_prob.reindex(sales.index.get_level_values(0))

q_means[2] = np.zeros(len(timeframes))
q_variances[2] = np.zeros(len(timeframes))

for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales.iloc[:, i].to_frame(),
        sample_weight=weights.iloc[:, i],
    )
    q_means[2][i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_variances[2][i] = mean_squared_error(sales.iloc[:, i], y) * len(fares)

print("Forecast Mean:", q_means[2])
print("Fcst Variance:", q_variances[2])

Forecast Mean: [11.6605633   1.9381824   1.65515212]
Fcst Variance: [34.24086023  1.19706163  0.50358652]


We can see from the max weights that any expected or observed `Y0` activity
that emerges in the first timeframe is going to overwhelmly control the
results for that timeframe.

In [22]:
weights.max()

TF
21    3250.997354
14      13.029445
7        2.420065
dtype: float64

### Option 3: LR Weighted by *Square* of Inverse Sellup Probability

This weights each equivalent unit of "Q" demand equally, 
counting more heavily the demand that survives when forced
to buy at a higher price. This gives even more weight to fitting 
demand at the higher fare classes, which have less activity 
but are more meaningful when they are active.

Without some kind of cap, this can end up giving VERY large 
weights to high value fare classes in early timeframes. In 
PODS there is a "max-cap" parameter, typically set to 10.0, 
which limits the up-scaling of historical demand.  A mathematically
equivalent `max_cap` is shown here.

This option is mathematically equivalent to the Q-forecasting
algorithm implemented in PODS (at least with respect to the 
forecast means), where all historical sales are converted to 
Q-equivalent sales, and forecasting is done on the Q-equivalent 
sales.

*Note: this option is not recommended, but provided here to
illustrate and contrast with other methods.*

In [23]:
max_cap = 10

weights = 1 / sellup_prob.reindex(sales.index.get_level_values(0)) ** 2
weights = weights.clip(0, max_cap**2)

q_means[3] = np.zeros(len(timeframes))
q_variances[3] = np.zeros(len(timeframes))


def max_cap_clip(x):
    """Clip the minumum of nonzero x to inverse of max_cap."""
    if x == 0:
        return 0
    return np.clip(x, 1.0 / max_cap, 1.0)


for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].apply(max_cap_clip).to_frame(),
        sales.iloc[:, i].to_frame(),
        sample_weight=weights.iloc[:, i],
    )
    q_means[3][i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_variances[3][i] = mean_squared_error(sales.iloc[:, i], y) * len(fares)

print("Forecast Mean:", q_means[3])
print("Fcst Variance:", q_variances[3])

Forecast Mean: [10.83738941  2.04752026  1.68058334]
Fcst Variance: [35.53059669  1.21788619  0.50449993]


The variance output from conditional forecasting is slightly different from the 
variance output from unconditional forecasters (e.g. "Q" forecast variance).  Here,
we have forecasted the variance conditional on knowing the lowest available fare class.
The naive "Q" forecast variance does not incorporate knowledge of the lowest available 
fare class, so it is simply the unconditional variance of the Q-equivalent sales.

Put another way: the naive forecaster variance is the total sum of squares, while 
the conditional forecaster variance is the linear regression's residual sum of squares.
In effect, some of the total variance observed may be attributable not only to "true"
variance in demand, but to variance introduced by the RM system manipulating the 
fare class availability.  In theory, if we assume the Frat5 curve is specified 
correctly, the RM system manipulation should not alter the observed variance, but
an misspecification of the sellup probabilities will end up being reflected here. 

In [24]:
naive_q_variance = (sales * np.sqrt(weights.values)).groupby("SAMPLE").sum().var()
naive_q_variance

TF
21    61.227067
14     3.420050
7      0.805810
dtype: float64

### Option 4: LR Weighted by Fare

Alternatively, we can balance between Options 1 and 2 by weighting
by fares.  This removes the problem of excessively large weights
from Option 2, but still tilts the result away from the unweighted
solution that implicitly overweights lower fareclasses.

Note: this option has no derivation from choice theory or other 
assumtions about the mathematical structure of processes, but is simply 
a conjecture about a possible value-based heuristic.

In [25]:
q_means[4] = np.zeros(len(timeframes))
q_variances[4] = np.zeros(len(timeframes))

for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales.iloc[:, i].to_frame(),
        sample_weight=fares.reindex(sales.index.get_level_values(0)),
    )
    q_means[4][i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_variances[4][i] = mean_squared_error(sales.iloc[:, i], y) * len(fares)

print("Forecast Mean:", q_means[4])
print("Fcst Variance:", q_variances[4])

Forecast Mean: [12.26926131  1.906933    1.66088499]
Fcst Variance: [33.84017763  1.1938125   0.50372458]


### Possible Research Question

The above solves an independent linear regression to get forecast "Q" demand for each 
timeframe. This assumes historical sales data from other timeframes does not have
useful information.  But a multi-target regression is readily possible:

In [26]:
m = LinearRegression(fit_intercept=False).fit(expected_sales_per_unit_demand, sales)
m.coef_

array([[13.0534334 , -2.18715886,  0.29015529],
       [-0.02898819,  1.82765287,  0.20351878],
       [-0.01363406,  0.03766003,  1.63465294]])

Our prior regression results essentially assume all the off-diagonal terms are zero.

Is this relaxation of the independent demand assumption useful?

### Q Forecast Partitioned

The mean and variance of the base "Q" demand implied by the 
coefficients of the linear regression in each timeframe are 
then partitioned to the various fare classes, according to 
the net sellup probabilties. This partitioning is needed to
support optimization methods that require forecasting by
fare class.

In [27]:
q_mean = q_means[2]
q_var = q_variances[2]

In [28]:
q_partitioned_mean_raw = net_sellup_prob * q_mean
q_partitioned_mean_raw

TF,21,14,7
FARECLASS,,,
Y0,0.003587,0.148754,0.683929
Y1,0.032566,0.161001,0.196463
Y2,0.328240,0.335259,0.252898
Y3,1.696923,0.473089,0.236297
Y4,4.482954,0.495352,0.184309
Y5,5.116293,0.324727,0.101257


In [29]:
q_partitioned_var_raw = net_sellup_prob * q_var
q_partitioned_var_raw

TF,21,14,7
FARECLASS,,,
Y0,0.010532,0.091874,0.208088
Y1,0.095628,0.099438,0.059774
Y2,0.963867,0.207063,0.076945
Y3,4.982959,0.292190,0.071894
Y4,13.164047,0.305940,0.056077
Y5,15.023827,0.200558,0.030808


In [30]:
# These mean and variance levels are zero'ed out whenever the adjusted fare is negative,
# and as appropriate for AP closures.
q_partitioned_mean = q_partitioned_mean_raw.where(fare_adj_ki > 0, 0)
for fareclass, ap in fare_ap.items():
    q_partitioned_mean.loc[fareclass, ap:] = 0
q_partitioned_mean

TF,21,14,7
FARECLASS,,,
Y0,0.003587,0.148754,0.683929
Y1,0.032566,0.161001,0.196463
Y2,0.328240,0.335259,0.000000
Y3,1.696923,0.473089,0.000000
Y4,4.482954,0.495352,0.000000
Y5,5.116293,0.000000,0.000000


In [31]:
q_partitioned_var = q_partitioned_var_raw.where(fare_adj_ki > 0, 0)
for fareclass, ap in fare_ap.items():
    q_partitioned_var.loc[fareclass, ap:] = 0
q_partitioned_var

TF,21,14,7
FARECLASS,,,
Y0,0.010532,0.091874,0.208088
Y1,0.095628,0.099438,0.059774
Y2,0.963867,0.207063,0.000000
Y3,4.982959,0.292190,0.000000
Y4,13.164047,0.305940,0.000000
Y5,15.023827,0.000000,0.000000


### Total Forecast to Departure

Take the cumulative sum (backward) for the forecast mean and variance 
to get the total forecast mean and variance to departure.

In [32]:
total_forecast_mean_to_departure = (
    q_partitioned_mean.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
)
total_forecast_mean_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.836270,0.832683,0.683929
Y1,0.390030,0.357464,0.196463
Y2,0.663499,0.335259,0.000000
Y3,2.170012,0.473089,0.000000
Y4,4.978307,0.495352,0.000000
Y5,5.116293,0.000000,0.000000


In [33]:
total_forecast_var_to_departure = (
    q_partitioned_var.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
)
total_forecast_var_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.310494,0.299962,0.208088
Y1,0.254840,0.159212,0.059774
Y2,1.170929,0.207063,0.000000
Y3,5.275149,0.292190,0.000000
Y4,13.469987,0.305940,0.000000
Y5,15.023827,0.000000,0.000000


In [34]:
total_forecast_stdev_to_departure = np.sqrt(total_forecast_var_to_departure)
total_forecast_stdev_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.557220,0.547688,0.456167
Y1,0.504817,0.399014,0.244488
Y2,1.082095,0.455041,0.000000
Y3,2.296769,0.540546,0.000000
Y4,3.670148,0.553118,0.000000
Y5,3.876058,0.000000,0.000000


# Conditional Forecast (Muddled)

All the above shows forecasting in a situation where all observed sales in each timeframe 
occur within the same fare class. However, there are many times when an RM system may close
or open a fare class during a timeframe, resulting in situations where the lowest
available fare class (where sales happen) is non-homogeneous.

In [35]:
# Muddled Sales Data
#
# This sales history represents a scenario where sometimes a fare class is
# closed during a timeframe, so that the lowest available fare class changes
# and actual sales may occur in multiple fare classes.  The "closures" data
# here is still stored as boolean, but the interpretation is more limited:
# a closure marker indicates that the fare class is closed for the entire
# timeframe, but unmarked fare classes are not necessarily open for the
# entire timeframe.

# fmt: off

sales_muddled, closures_muddled = format_sales_and_closure(
    Y5=[
        [ 7, X, X],  # 00
        [ 7, X, X],  # 01
        [ 6, X, X],  # 02
        [ X, X, X],  # 03
        [19, X, X],  # 04
        [ X, X, X],  # 05
        [11, X, X],  # 06
        [ X, X, X],  # 07
        [17, X, X],  # 08
        [ 8, X, X],  # 09
        [ X, X, X],  # 10
        [23, X, X],  # 11
        [28, X, X],  # 12
        [17, X, X],  # 13
        [13, X, X],  # 14
        [ X, X, X],  # 15
        [11, X, X],  # 16
        [18, X, X],  # 17
        [ X, X, X],  # 18
        [ X, X, X],  # 19
        [ X, X, X],  # 20
        [ X, X, X],  # 21
        [ X, X, X],  # 22
        [ X, X, X],  # 23
        [ X, X, X],  # 24
        [ X, X, X],  # 25
    ],
    Y4=[
        [2, 1, X],  # 00
        [1, 1, X],  # 01
        [0, 1, X],  # 02
        [3, 1, X],  # 03
        [0, 2, X],  # 04
        [5, 1, X],  # 05
        [0, 1, X],  # 06
        [6, 2, X],  # 07
        [0, 1, X],  # 08
        [0, 1, X],  # 09
        [9, X, X],  # 10
        [2, 1, X],  # 11
        [4, X, X],  # 12
        [0, 1, X],  # 13
        [0, 2, X],  # 14
        [4, X, X],  # 15
        [0, 2, X],  # 16
        [3, 1, X],  # 17
        [0, 2, X],  # 18
        [0, 1, X],  # 19
        [0, 1, X],  # 20
        [9, 2, X],  # 21
        [0, 1, X],  # 22
        [X, X, X],  # 23
        [0, 1, X],  # 24
        [X, X, X],  # 25
    ],
    Y3=[
        [0, 0, 1],  # 00
        [0, 0, 2],  # 01
        [0, 0, 0],  # 02
        [0, 0, 1],  # 03
        [0, 1, X],  # 04
        [0, 0, 1],  # 05
        [0, 0, 3],  # 06
        [0, 0, 2],  # 07
        [0, 0, 1],  # 08
        [0, 0, 2],  # 09
        [0, 1, 3],  # 10
        [0, 1, 1],  # 11
        [0, 5, X],  # 12
        [0, 0, 1],  # 13
        [0, 0, 2],  # 14
        [0, 2, 1],  # 15
        [0, 0, 1],  # 16
        [0, 1, X],  # 17
        [0, 0, X],  # 18
        [0, 0, 1],  # 19
        [0, 0, X],  # 20
        [0, 0, X],  # 21
        [0, 0, 0],  # 22
        [X, X, 1],  # 23
        [0, 0, X],  # 24
        [4, 4, 1],  # 25
    ],
    Y2=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, X],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [1, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, X],  # 17
        [0, 0, 1],  # 18
        [0, 0, 0],  # 19
        [0, 0, 2],  # 20
        [0, 0, 1],  # 21
        [0, 0, 1],  # 22
        [1, 0, 0],  # 23
        [0, 0, 1],  # 24
        [0, 0, 1],  # 25
    ],
    Y1=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 1],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, X],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 1],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 1],  # 25
    ],
    Y0=[
        [0, 0, 0],  # 00
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 1],  # 12
        [0, 0, 0],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
    ],
)

# fmt: on

The question now becomes: how do we handle partial closures?

In [36]:
q_mud_means = {}
q_mud_variances = {}

## Option 11: Ignore Muddling

We can push this sales data through the identical algorithm, and simply ignore the partial closures.

In [37]:
top_class = fares.index[0]

lowest_available_class_muddled = pd.DataFrame(
    0,
    columns=closures_muddled.loc[top_class].columns,
    index=closures_muddled.loc[top_class].index,
)
for class_number, fare_class in enumerate(fares.index[1:], start=1):
    lowest_available_class_muddled = lowest_available_class_muddled.where(
        closures_muddled.loc[fare_class], class_number
    )

expected_sales_muddled_per_unit_demand = expected_sales_func(
    lowest_available_class_muddled
)

In [ ]:
q_mud_means[11] = np.zeros(len(timeframes))
q_mud_variances[11] = np.zeros(len(timeframes))

for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_muddled_per_unit_demand.iloc[:, i].to_frame(),
        sales_muddled.iloc[:, i].to_frame(),
    )
    q_mud_means[11][i] = m.coef_[0][0]
    y = m.predict(expected_sales_muddled_per_unit_demand.iloc[:, i].to_frame())
    q_mud_variances[11][i] = mean_squared_error(sales_muddled.iloc[:, i], y) * len(
        fares
    )

print("Forecast Mean:", q_mud_means[11])
print("Fcst Variance:", q_mud_variances[11])


Forecast Mean: [12.48363676  1.83805027  1.6379777 ]
Fcst Variance: [35.15717185  1.30627819  0.61879392]


## Option 12: Approximate Closures

Given some observed sales in what appears to be an other-than-lowest fare class, we can 
use the sellup probabilities and the relative numbers of sales to approximate the 
fraction of time within each timeframe that each fare class was the lowest available
fare class.

The basic logic is as follows:

- For historical timeframes with some sales, all of which are in a single fare class,
  we assume that the relevant fare class was the lowest class available for the entire
  timeframe, even if that fare class is not recorded as the apparent lowest class 
  available.
- For historical timeframes with some sales, but not all in the same fare class, we
  assume that each sold fareclass was the lowest fare class for a fraction of the 
  timeframe proportional to that fare class's observed sales, weighted by the inverse
  of the sellup probability for the fare class.
- For historical timeframes with no sales, we assume the recorded lowest class available
  was the actual lowest class available for the entire timeframe.  Mechanically, in this
  example this is achieved by adding a tiny epsilon of sales to the recorded lowest class,
  which is a meaningless weight compared to actual sales, but enough to normalize to 1
  in the absence of any real sales.

In the example code below, the `inferred_lowest_class` is an indicator DataFrame, where
cells take a value of 1.0 when they are always the lowest available class, some non-zero
fraction when they are sometimes the lowest class, or zero when they are never the 
lowest class.

In [ ]:
lowest_available_class_marker = (expected_sales_muddled_per_unit_demand > 0) * 1e-99


def normalize(x):
    return x / x.sum()


q_inflation_factors = 1.0 / sellup_prob

inferred_lowest_class = (
    (sales_muddled * q_inflation_factors + lowest_available_class_marker)
    .groupby("SAMPLE")
    .transform(normalize)
)
inferred_lowest_class

TF                      21   14   7 
FARECLASS SAMPLE                    
Y5        0       0.662653  0.0  0.0
          1       0.797103  0.0  0.0
          2       1.000000  0.0  0.0
          3       0.000000  0.0  0.0
          4       1.000000  0.0  0.0
...                    ...  ...  ...
Y0        21      0.000000  0.0  0.0
          22      0.000000  0.0  0.0
          23      0.000000  0.0  0.0
          24      0.000000  0.0  0.0
          25      0.000000  0.0  0.0

[156 rows x 3 columns]

Given this indicator formated DataFrame, the computation of the expected sales per unit demand
is much simpler, as we can simply multiply the `inferred_lowest_class` indicator against
the `sellup_prob` data, and pandas will align the data automatically.

In [40]:
expected_sales_per_unit_demand_option_12 = inferred_lowest_class * sellup_prob

Now we can feed these expected sales into the usual linear regression to solve for
the forecast "Q" mean and conditional variance.

In [ ]:
q_mud_means[12] = np.zeros(len(timeframes))
q_mud_variances[12] = np.zeros(len(timeframes))

for i in range(len(timeframes)):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand_option_12.iloc[:, i].to_frame(),
        sales_muddled.iloc[:, i].to_frame(),
    )
    q_mud_means[12][i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand_option_12.iloc[:, i].to_frame())
    q_mud_variances[12][i] = mean_squared_error(sales_muddled.iloc[:, i], y) * len(
        fares
    )

print("Forecast Mean:", q_mud_means[12])
print("Fcst Variance:", q_mud_variances[12])


Forecast Mean: [13.05507655  1.9520325   1.7559949 ]
Fcst Variance: [39.93497886  1.23429957  0.46775185]
